In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
rsv = pd.read_csv('/kaggle/input/goodreadsbooks/books.csv', error_bad_lines=False)

In [ ]:
rsv.head()

In [ ]:
fn = rsv.authors.unique()
thresh = 20
writers = dict()
for f in fn:
    fc = rsv[rsv.authors == f].bookID.count() 
    if fc >= thresh: 
        writers[f] = fc

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
sum(writers.values())/len(writers)

In [ ]:
writers = dict(sorted(writers.items(), key=lambda item: item[1]))

In [ ]:
plt.figure(figsize=(40, 40))
plt.bar(range(len(writers)), sorted(list(writers.values()))[::-1], align='center')
plt.xticks(range(len(writers)), list(writers.keys()), fontsize=30, rotation=90)
plt.show();

In [ ]:
title_arr = []
writers_arr = []
for f in fn:
    if '/' in f:
        try:
            if title_arr[-1] == rsv[rsv.authors == f].title.values[0]:
                if len(writers_arr[-1]) < len(f.split('/')):
                    writers_arr[-1] = f.split('/')
            else:
                title_arr.append(rsv[rsv.authors == f].title.values[0])
                writers_arr.append(f.split('/'))
        except IndexError:
            title_arr.append(rsv[rsv.authors == f].title.values[0])
            writers_arr.append(f.split('/'))

In [ ]:
unique_writes = set()
count = 0
max_len = 0
for x,y  in zip(title_arr, writers_arr):
    if len(y) > max_len:
        max_len = len(y)
    for z in y:
        unique_writes.add(z)
        count+=1

In [ ]:
len(unique_writes)

In [ ]:
count

In [ ]:
get_books = dict.fromkeys(unique_writes, 0)

In [ ]:
for x,y in zip(title_arr, writers_arr):
    for z in y:
        get_books[z]+=1

In [ ]:
rsv.sort_values(by=['ratings_count'],ascending=False)[:10]

In [ ]:
get_books = dict(sorted(get_books.items(), key=lambda item: item[1])[::-1])

In [ ]:
plt.figure(figsize=(40, 40));
plt.bar(range(10), list(get_books.values())[:10], align='center')
plt.xticks(range(10), list(get_books.keys())[:10], fontsize=30, rotation=90)
plt.yticks(range(50), range(50), fontsize=30)
plt.show();

In [ ]:
get_books.